# Classification des points (points isolés et points sol)

Quand nous travaillons sur un nuage de points, il est intéressant de pouvoir attribuer une classe à un point en fonction de sa nature. Ainsi, c'est ce point représente le sol, il se verra attribuer la classe 2 Ground. Ensuite en ne gardant que les points de la classe 2 ground, nous pourrions créer un MNT.

PDAL a plusieurs filters permmettant de classifier les points. https://pdal.io/en/latest/stages/filters.html


<img align="left" src="data/img/pdal_ground.png">

In [ ]:
%%time
import pathlib, json, pdal

#chemin du fichier source
datasource = pathlib.Path.cwd() /'data'/'input'
output = pathlib.Path.cwd() /'data'/'output'
file = '2560_1145.las'

#pipeline pdal
classification = {
 "pipeline":[
     
    #lecture du fichier source
     {
        "type" : "readers.las",
        "filename" : str(datasource/file)
    },
     
    #déclassification du nuage car il était déjà classifié
    {
        "type":"filters.assign",
        "assignment":"Classification[:]=1"
    },
     
    #identifie les low points comme bruit
    {
        "type":"filters.elm"
    },
     
    #attribue la classe 7 aux points identifiés précédemment comme bruit
    {
        "type":"filters.outlier"
    }, 
     
    #classification du ground en utilisant Pinguel2013
    {
        "type":"filters.smrf",
        "ignore":"Classification[7:7]",
        "slope":0.2,
        "window":16,
        "threshold":0.45,
        "scalar":1.2
    },     
     
    #enregistrement du nuage
    {   "type":"writers.las",
        "filename": str(output)+"\\"+"cloud_classified.las"
    }
]   
}

pipeline = pdal.Pipeline(json.dumps(classification))
pipeline.execute()

La cellule peut prendre **plusieurs minutes** à s'éxécuter en raison de la complexité des algorithmes utilisés.